In [2]:
#!pip install torch transformers bitsandbytes accelerate peft vllm

In [4]:
#pip install openpyxl

In [6]:
#pip install openai

In [5]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import pandas as pd
from openai import OpenAI
import json
import time
import warnings
warnings.filterwarnings('ignore')

WARNING 12-18 12:00:11 _custom_ops.py:18] Failed to import from vllm._C with ModuleNotFoundError("No module named 'vllm._C'")
INFO 12-18 12:00:11 importing.py:10] Triton not installed; certain GPU-related functions will not be available.


In [11]:
API_KEY=""""""
client = OpenAI(
    api_key=API_KEY,
)
MAX_RETRIES = 3
SLEEP_DURATION = 5

In [9]:
model_id = "Open-Orca/Mistral-7B-OpenOrca"
llm = LLM(model=model_id, max_model_len=16240)

RuntimeError: Failed to infer device type

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [14]:
def chatGPT4oresponse(ques):               
    
    for _ in range(MAX_RETRIES):
        try:
            response = client.chat.completions.create(
                model='gpt-4o',
                messages=[
                    {"role": "user", "content": "{}".format(ques)},
                ],
                temperature=0,
            )
            return(response.choices[0].message.content)
        except Exception as e:
            if _ == MAX_RETRIES - 1:  # Last iteration
                error = "An error occurred: {}\n".format(e)
                return error
            time.sleep(SLEEP_DURATION)  # Sleep before retrying

def base_gpt4_model(ques):
    for _ in range(MAX_RETRIES):
        try:
            response = client.chat.completions.create(
                model='gpt-4-turbo',
                messages=[
                    {"role": "user", "content": "{}".format(ques)},
                ],
                temperature=0,
            )
            return(response.choices[0].message.content)
        except Exception as e:
            if _ == MAX_RETRIES - 1:  # Last iteration
                return "An error occurred: {}".format(e)
            time.sleep(SLEEP_DURATION)  # Sleep before retrying

def generate_terraform_code(user_query,cloud_provider):
    instruction = "Generate Terraform code based on the provided requirements."
    prompt="""You are a Terraform consultant with expertise in AWS, Azure, and GCP. Your task is to develop a Terraform configuration based on the provided user query for the specified cloud provider.

    - *User Query:* {}
    - *Cloud Provider:* {}

    This Terraform configuration must include modules for essential components and design the modules to be reusable and easy to maintain.
    Adhere to the industry best practices for security and efficiency throughout the configuration. The output Terraform config should be in a structured format that is easy to understand and implement."""
    updated_prompt=prompt.format(user_query,cloud_provider)
    chat = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": updated_prompt}
    ]
    
    formatted_prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    return formatted_prompt

In [15]:
df=pd.read_excel('Research_Analysis_Base_data_filtered_new.xlsx')
del df['Unnamed: 0']

In [16]:
df.head()

,Category,Cloud_provider,user_query,Terraform_Code
0,GoogleCloudPlatform,GCP,Please generate a Terraform configuration for ...,#\n# Copyright 2020 Google LLC\n#\n# Licensed ...
1,immutability-io,AWS,Please generate a Terraform configuration for ...,# DHCP Options\n\n## Set Terraform version con...
2,forseti-security,GCP,Please generate a Terraform configuration for ...,/**\n * Copyright 2019 Google LLC\n *\n * Lice...
3,cncf,AWS,Please generate a Terraform configuration for ...,"# resource ""azurerm_network_security_group"" ""c..."
4,terraform-google-modules,GCP,Please generate a Terraform configuration for ...,"resource ""google_compute_ssl_certificate"" ""def..."


In [17]:
#df=df.head(10)

In [18]:
prompt_lst=[]
for index, row in df.iterrows():
    user_query=row['user_query']
    cloud_provider=row['Cloud_provider']
    #print("user query is {}".format(user_query))
    try:
        updated_prompt=generate_terraform_code(user_query,cloud_provider)
        prompt_lst.append(updated_prompt)
    except Exception as error:
        print(error)

In [19]:
prompt_lst[0]

'<|im_start|>system\nGenerate Terraform code based on the provided requirements.<|im_end|>\n<|im_start|>user\nYou are a Terraform consultant with expertise in AWS, Azure, and GCP. Your task is to develop a Terraform configuration based on the provided user query for the specified cloud provider.\n\n    - *User Query:* Please generate a Terraform configuration for setting up a Google Cloud Platform (GCP) project with necessary services, a VPC network, subnets, firewall rules, a NAT router, compute instances, instance templates, health checks, an instance group manager, and a global load balancer.\n    - *Cloud Provider:* GCP\n\n    This Terraform configuration must include modules for essential components and design the modules to be reusable and easy to maintain.\n    Adhere to the industry best practices for security and efficiency throughout the configuration. The output Terraform config should be in a structured format that is easy to understand and implement.<|im_end|>\n<|im_start|

In [20]:
sampling_params = SamplingParams(temperature=0.7, top_p=0.95, max_tokens=2500, stop=['[/SYS]', "<s>", "</s>","<|im_end|>"])
outputs= llm.generate(prompt_lst, sampling_params)
terraform_code = [output.outputs[0].text.strip() for output in outputs]

Processed prompts:   0%|          | 0/461 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

WARNING 08-11 13:56:45 scheduler.py:1099] Sequence group 170 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1
WARNING 08-11 13:56:52 scheduler.py:1099] Sequence group 120 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=51


Processed prompts:   1%|          | 3/461 [00:25<48:24,  6.34s/it, est. speed input: 22.25 toks/s, output: 26.96 toks/s]  

WARNING 08-11 13:57:05 scheduler.py:1099] Sequence group 69 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=101


Processed prompts:  13%|█▎        | 61/461 [01:28<04:25,  1.51it/s, est. speed input: 131.88 toks/s, output: 498.42 toks/s]

...

Processed prompts:  86%|████████▋ | 398/461 [11:47<01:33,  1.49s/it, est. speed input: 109.07 toks/s, output: 592.69 toks/s]

WARNING 08-11 14:08:30 scheduler.py:1099] Sequence group 430 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1001


Processed prompts:  91%|█████████ | 419/461 [12:36<01:07,  1.61s/it, est. speed input: 107.49 toks/s, output: 592.34 toks/s]

WARNING 08-11 14:09:15 scheduler.py:1099] Sequence group 460 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1051


Processed prompts: 100%|██████████| 461/461 [14:10<00:00,  1.84s/it, est. speed input: 105.40 toks/s, output: 588.84 toks/s]


In [21]:
df['Generated_Tearraform_Code_Mistral_7B']=terraform_code

In [30]:
print(df.iloc[4]['Generated_Tearraform_Code_Mistral_7B'])

terraform {
  required_providers {
    google = {
      source = "hashicorp/google"
      version = "4.51.0"
    }
  }
}

provider "google" {
  project = "your-project-id"
  region  = "your-region"
}

resource "random_id" "suffix" {
  byte_length = 4
}

module "ssl_certificate" {
  source = "your-module-path/ssl-certificate"

  name_prefix = "your-prefix"
  description = "your-description"
  random_suffix = random_id.suffix.dec
}

output "ssl_certificate_arn" {
  value = module.ssl_certificate.ssl_certificate_arn
}


In [27]:
df.head()

,Category,Cloud_provider,user_query,Terraform_Code,Generated_Tearraform_Code_Mistral_7B
0,GoogleCloudPlatform,GCP,Please generate a Terraform configuration for ...,#\n# Copyright 2020 Google LLC\n#\n# Licensed ...,# Google Cloud Platform (GCP) Terraform Config...
1,immutability-io,AWS,Please generate a Terraform configuration for ...,# DHCP Options\n\n## Set Terraform version con...,terraform {\n required_providers {\n aws =...
2,forseti-security,GCP,Please generate a Terraform configuration for ...,/**\n * Copyright 2019 Google LLC\n *\n * Lice...,# Terraform Configuration for Google Cloud Pub...
3,cncf,AWS,Please generate a Terraform configuration for ...,"# resource ""azurerm_network_security_group"" ""c...",#---------------------------------------------...
4,terraform-google-modules,GCP,Please generate a Terraform configuration for ...,"resource ""google_compute_ssl_certificate"" ""def...",terraform {\n required_providers {\n googl...


In [24]:
evaluvate_terraform_prompt="""You are an expert Terraform developer specializing in infrastructure as code for cloud environments. I need you to evaluate the given Generated Terraform codes based on a user query and Golden Terraform code to determine whether the generated Terraform codes are correct or not. For each Terraform code, ensure the following:
Correctness: Verify if the code meets the user’s requirements as described in the query.
Best Practices: Check if the code follows best practices for the chosen cloud provider.
Completeness: Ensure all necessary resources and configurations are included.

User query :{}
Golden Terraform Code : {}
Generated terraform Code : {}
The output should contain whether the generated code is  correct or not for the user query and Golden Terraform code , only validate it logically and functionally 
example output :  Correct:Yes/No, Reason : <Add reason>
"""

In [25]:
df.to_excel("Research_Analysis_Base_data_filtered_new_Mistral_7B.xlsx")

In [26]:
df.shape

(461, 5)

In [ ]:
# Define the function to determine the cloud provider
def evaluvate_terraform(inp):
    golden=inp['Terraform_Code']
    user_query=inp['user_query']
    generated=inp['Generated_Tearraform_Code_Mistral_7B']
    updated_prompt=evaluvate_terraform_prompt.format(user_query,golden,generated)
    res=chatGPT4oresponse(updated_prompt)
    return (res)

# Apply the function only where Cloud_provider is 'NVF'
df['Validation_Result'] = df.apply(evaluvate_terraform,axis=1)